In [1]:
import geopandas as gpd
import rasterio.features
import shapely

from roofsense.bag3d import BAG3DTileStore, LevelOfDetail

In [2]:
tile_store = BAG3DTileStore(dirpath=r"data/store")
tile_id = "10-280-560"

In [15]:
for name, lod in zip(
    ["lod12", "lod13", "lod22"],
    [LevelOfDetail.LoD12, LevelOfDetail.LoD13, LevelOfDetail.LoD22],
):
    src_filepath = rf"../obia/{tile_id}.map.{lod}.tif"
    dst_filepath = rf"../test/{tile_id}.map.{lod}.test.tif"

    all_surfs = gpd.read_file(f"{tile_id}.test.{lod.name}.gpkg")

    src: rasterio.io.DatasetReader
    with rasterio.open(src_filepath) as src:
        bbox = gpd.GeoDataFrame(
            {"id": [0], "geometry": [shapely.box(*src.bounds)]}, crs="EPSG:28992"
        )
        data = src.read(indexes=1)
        meta = src.profile

    valid_surfs = all_surfs.overlay(bbox)

    rasterio.features.rasterize(
        shapes=(
            (surf, val)
            for surf, val in zip(valid_surfs.geometry, valid_surfs._majority)
        ),
        out_shape=src.shape,
        transform=src.transform,
    )

    dst: rasterio.io.DatasetWriter
    with rasterio.open(dst_filepath, mode="w", **meta) as dst:
        dst.write(data, indexes=1)